# MEX INEGI 2020 Population and Housing Census

This dataset contains the data from the population and housing census performed in Mexico on the year 2020. 

### Goal:
* Fetch selected columns from the csv file, preprocess the data and export as a new csv file to be inserted into a SQL Server table (location).

In [1]:
# Import the relevant libraries
import pandas as pd
import numpy as np
import unicodedata
import re

In [2]:
"""
Read csv file from INEGI website and display its first rows
* URL: https://www.inegi.org.mx/datosabiertos/

The route to acces the data is the following:

Informacion Demografica y Social > Censos y Conteos > Censos y Conteos de Poblacion y vivienda >
2020 > Principales resultados por localidad (ITER) > Estados Unidos Mexicanos
"""

raw_data = pd.read_csv('conjunto_de_datos_iter_00CSV20.csv', low_memory=False)
df = raw_data.copy()
df.head()

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,LONGITUD,LATITUD,ALTITUD,POBTOT,...,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC,TAMLOC
0,0,Total nacional,0,Total nacional,0,Total nacional,NaN,NaN,NaN,126014024,...,30775898,18307193,15211306,6616141,4047100,1788552,3170894,15108204,852871,*
1,0,Total nacional,0,Total nacional,9998,Localidades de una vivienda,NaN,NaN,NaN,250354,...,47005,8385,18981,1732,1113,12775,14143,51293,7154,*
2,0,Total nacional,0,Total nacional,9999,Localidades de dos viviendas,NaN,NaN,NaN,147125,...,25581,5027,11306,971,708,8247,10065,29741,5283,*
3,1,Aguascalientes,0,Total de la entidad Aguascalientes,0,Total de la Entidad,NaN,NaN,NaN,1425607,...,359895,236003,174089,98724,70126,6021,15323,128996,1711,*
4,1,Aguascalientes,0,Total de la entidad Aguascalientes,9998,Localidades de una vivienda,NaN,NaN,NaN,3697,...,732,205,212,48,41,39,62,530,20,*


In [3]:
# Display the amount of rows and columns of the dataframe
nr_rows = df.shape[0]
nr_col = df.shape[1]
print(f'There are {nr_rows} rows and {nr_col} columns in the dataframe.')

There are 195662 rows and 286 columns in the dataframe.


In [4]:
# Get a list of the column names to select the needed ones 
# list(df.columns)

### Create First DataFrame ("location" table)

In this section we will select the columns for the "location" dataframe. This dataframe will be exported as a separate csv file and further turned into a SQL Server table.
This dataframe will contain data about the territory distribution in Mexico (state, municipality and locality) and geographical data of each localty.

In [5]:
# Select the relevant columns
df_location = df[['ENTIDAD','NOM_ENT','MUN','NOM_MUN','LOC', 'NOM_LOC', 'LONGITUD','LATITUD']]
df_location.head()

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,LONGITUD,LATITUD
0,0,Total nacional,0,Total nacional,0,Total nacional,NaN,NaN
1,0,Total nacional,0,Total nacional,9998,Localidades de una vivienda,NaN,NaN
2,0,Total nacional,0,Total nacional,9999,Localidades de dos viviendas,NaN,NaN
3,1,Aguascalientes,0,Total de la entidad Aguascalientes,0,Total de la Entidad,NaN,NaN
4,1,Aguascalientes,0,Total de la entidad Aguascalientes,9998,Localidades de una vivienda,NaN,NaN


In [6]:
# Remove accents and special characters

def remove_accents(input_str):
    if isinstance(input_str, str):
        nfkd_form = unicodedata.normalize('NFKD', input_str)
        return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])
    return input_str

# Apply to all string cells in the dataframe
df_noaccent = df_location.map(remove_accents)
df_noaccent.head(10)

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,LONGITUD,LATITUD
0,0,Total nacional,0,Total nacional,0,Total nacional,NaN,NaN
1,0,Total nacional,0,Total nacional,9998,Localidades de una vivienda,NaN,NaN
2,0,Total nacional,0,Total nacional,9999,Localidades de dos viviendas,NaN,NaN
3,1,Aguascalientes,0,Total de la entidad Aguascalientes,0,Total de la Entidad,NaN,NaN
4,1,Aguascalientes,0,Total de la entidad Aguascalientes,9998,Localidades de una vivienda,NaN,NaN
5,1,Aguascalientes,0,Total de la entidad Aguascalientes,9999,Localidades de dos viviendas,NaN,NaN
6,1,Aguascalientes,1,Aguascalientes,0,Total del Municipio,NaN,NaN
7,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,"102°17'45.768"" W","21°52'47.362"" N"
8,1,Aguascalientes,1,Aguascalientes,94,Granja Adelita,"102°22'24.710"" W","21°52'18.749"" N"
9,1,Aguascalientes,1,Aguascalientes,96,Agua Azul,"102°21'25.639"" W","21°53'01.522"" N"


In [7]:
# Function to convert coordinate values into Tableau compatible numbers

def dms_to_decimal_degrees(dms_str):
    """
    Converts a Degrees, Minutes, Seconds (DMS) string to decimal degrees,
    rounded to 4 decimal places. Returns None if the input is 'NULL' or not in a valid format.
    Example DMS string: "111°40'24.022\" W" or "25°02'04.688\" N"
    """
    if not isinstance(dms_str, str) or dms_str.strip().upper() == 'NULL':
        return None

    dms_str = dms_str.strip()

    # Flexible regex: allows optional whitespace and optional degree/minute/second symbols
    pattern = r"""^\s*(\d+)[°\s]*(\d+)[\'\s]*([\d.]+)["\s]*([NSWE])\s*$"""
    match = re.match(pattern, dms_str, re.IGNORECASE)

    if not match:
        return None  # More robust than raising an exception

    degrees = float(match.group(1))
    minutes = float(match.group(2))
    seconds = float(match.group(3))
    direction = match.group(4).upper()

    decimal_degrees = degrees + (minutes / 60) + (seconds / 3600)

    if direction in ['S', 'W']:
        decimal_degrees *= -1

    return round(decimal_degrees, 4)


In [8]:
# Create a copy of the dataframe before running the conversion function
df_final = df_noaccent.copy()

# Now, apply the conversion
df_final['LONGITUD'] = df_final['LONGITUD'].apply(dms_to_decimal_degrees)
df_final['LATITUD'] = df_final['LATITUD'].apply(dms_to_decimal_degrees)

df_final.head(10)

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,LONGITUD,LATITUD
0,0,Total nacional,0,Total nacional,0,Total nacional,NaN,NaN
1,0,Total nacional,0,Total nacional,9998,Localidades de una vivienda,NaN,NaN
2,0,Total nacional,0,Total nacional,9999,Localidades de dos viviendas,NaN,NaN
3,1,Aguascalientes,0,Total de la entidad Aguascalientes,0,Total de la Entidad,NaN,NaN
4,1,Aguascalientes,0,Total de la entidad Aguascalientes,9998,Localidades de una vivienda,NaN,NaN
5,1,Aguascalientes,0,Total de la entidad Aguascalientes,9999,Localidades de dos viviendas,NaN,NaN
6,1,Aguascalientes,1,Aguascalientes,0,Total del Municipio,NaN,NaN
7,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,-102.2960,21.8798
8,1,Aguascalientes,1,Aguascalientes,94,Granja Adelita,-102.3735,21.8719
9,1,Aguascalientes,1,Aguascalientes,96,Agua Azul,-102.3571,21.8838


In [9]:
# View column data types
df_location.dtypes

ENTIDAD      int64
NOM_ENT     object
MUN          int64
NOM_MUN     object
LOC          int64
NOM_LOC     object
LONGITUD    object
LATITUD     object
dtype: object

In [10]:
# Export the dataframe as a new .csv file

# df.to_csv('output.csv', index=False, date_format='%Y-%m-%d %H:%M:%S', encoding='utf-8')
df_final.to_csv('location.csv', index=False, na_rep='', encoding='utf-8')
print("Successfully exported the dataframe as 'location.csv'")

Successfully exported the dataframe as 'location.csv'
